# Quickstart
This notebook serves as a quickstart into the **pyecsca** toolkit that you will be using during the tutorial.

In [ ]:
import numpy as np
import holoviews as hv

from pyecsca.ec.key_generation import KeyGeneration
from pyecsca.ec.key_agreement import ECDH_SHA1
from pyecsca.ec.signature import ECDSA_SHA1
from pyecsca.ec.params import get_params
from pyecsca.ec.mult import LTRMultiplier
from pyecsca.ec.context import DefaultContext, local
from pyecsca.ec.mod import Mod

from pyecsca.sca.trace import Trace
from pyecsca.sca.trace.plot import plot_trace, plot_traces

hv.extension("bokeh")
%opts RGB [height=600, responsive=True]

First, let's get some curves. 💃

The cell below loads the `secp256r1` curve in the `secg` category, in `projective` coordinates, from the store of standard curves [std-curves](https://github.com/J08nY/std-curves) included in **pyecsca**.

In [ ]:
p256 = get_params("secg", "secp256r1", "projective")
p256

You can examine the domain parameters, they consist of the curve, the generator, the order and cofactor.

In [ ]:
curve = p256.curve
generator = p256.generator
print(repr(curve))
print(repr(generator))
print(p256.order, p256.cofactor)

The curve has a model (Short-Weierstrass, Montgomery, Twisted Edwards or Edwards), some parameters and a prime defining the field.
The curve also has a coordinate model (a coordinate system, in this case homogenous projective).

In [ ]:
model = p256.curve.model
coords = p256.curve.coordinate_model
print(repr(model))
print(repr(coords))

The generator is a point on the curve, in projective coordinates.

In [ ]:
print(repr(generator))
affine_generator = generator.to_affine()
print(repr(affine_generator))

We can also do some basic calculations with the affine generator point.

In [ ]:
print("Times 5", curve.affine_multiply(affine_generator, 5))
print("Twice",   curve.affine_double(affine_generator))
print("Random",  curve.affine_random())
print("x=123",   curve.affine_lift_x(Mod(123, curve.prime)))

Or with individual finite-field elements.

In [ ]:
k = Mod(123, curve.prime)
l = Mod.random(curve.prime)
print(k, l)
print(k.inverse())
print(k.sqrt())
print(k + l)

Let's examine the curve model and coordinate model objects. These hold the data on coordinate systems and formulas extracted from the [EFD](https://www.hyperelliptic.org/EFD/).

In [ ]:
for name, coordinates in model.coordinates.items():
    print(f"{name:13}{coordinates!r}")
print("----------")
for name, formula in coords.formulas.items():
    print(f"{name:15}{formula.shortname:10}{formula}")

## Traces
There is also functionality to work with traces from side-channel measurement.

In [ ]:
rng = np.random.default_rng()
trace = Trace(rng.standard_normal(1000), meta={"stuff": "Some metadata here", "a": 1})
plot_trace(trace)

You can access the metadata of a trace:

In [ ]:
trace.meta

The samples of the trace are just an ordinary numpy array. You can thus use any numpy-adjacent libraries with it, like numpy or scipy.

In [ ]:
from scipy.signal import find_peaks

peaks, heights = find_peaks(trace.samples, height=2)
plot_trace(trace) * hv.Points((peaks, trace.samples[peaks])).opts(color="red")

There are also trace manipulation functions directly included in the toolkit, for example the absolute value

In [ ]:
absolute_trace = absolute(trace)
plot_trace(absolute_trace)

## Simulation
**pyecsca** is able to simulate computation of key generation, ECDH and ECDSA while tracing particular actions performed by the implementation as well as intermediate values. These traces are collected by the context (see the [Context](https://neuromancer.sk/pyecsca/api/pyecsca.ec.context.html#pyecsca.ec.context.Context) and [DefaultContext](https://neuromancer.sk/pyecsca/api/pyecsca.ec.context.html#pyecsca.ec.context.DefaultContext) classes).

These traces are useful for attacks which rely on computing particular intermediate values during the ECC computation, but also for reverse-engineering.

Let's pick some formulas and construct a basic left-to-right scalar multiplier.

![](../img/ltr.svg)

In [ ]:
add = coords.formulas["add-2007-bl"]
dbl = coords.formulas["dbl-2007-bl"]
scl = coords.formulas["z"]

mult = LTRMultiplier(add, dbl, scl)

Now we can instantiate a default context, initialize the multiplier and multiply with a picked scalar.

In [ ]:
with local(DefaultContext()) as ctx:
    mult.init(p256, generator)
    res = mult.multiply(123456789)

The result is available:

In [ ]:
res

We can also observe the actions taken while inside the context. They form a tree:

In [ ]:
tree = ctx.actions
tree

In [ ]:
scalarmult_action, subtree = tree.get_by_index([0])
subtree

## <span style="color:#00468C; font-weight: bold;">Exercise</span>
Now, you can try to look at the sequence of formula applications and extract the scalar from them, given that a simple left-to-right double-and-add scalarmult was used.

In [ ]:
recovered_privkey = 1
for formula_call in subtree:
    if formula_call.formula.shortname == "add":
        recovered_privkey |= 1
    elif formula_call.formula.shortname == "dbl":
        recovered_privkey <<= 1

print(recovered_privkey)